**IMPORT ALL THE LIBRARY**

In [0]:
import pandas as pd
import numpy as np
from scipy.io.wavfile import read
from scipy.signal import spectrogram
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import librosa

specify the directory paths

**all csv file names shold be same as audio files**

In [0]:
datadir_cutout = 'Project/word Boundary/word cutout/' # set the path of the csv files of the audio samples
datadir_waves =  'Project/word Boundary/all wave files/' # set the path of the wave files

CREATING TRAINING DATA

In [0]:
training_data=[]
train_x = []
train_y = []
for files in tqdm(os.listdir(datadir_cutout)):
    try:
        fs,audio = read(datadir_waves + files[:-3] +'wav') 
        n = len(audio)
        l = (((n//(fs//2))+1)*(fs//2))
        audio =np.append(audio,np.zeros((l-n)))
        df = pd.read_csv(datadir_cutout+files)
        time  = np.arange(0,len(audio)/fs,1/fs)
        m = min(len(time),len(audio))
        word = np.zeros((len(audio)))
        frame_word=[]
        frame_time=[]
        for i in range(df.shape[0]):
            data = list(df.loc[i])[0].split('\t')
            start , duration = float(data[1][2:]),float(data[2][2:])
            end = start+duration
            word[int(start*fs+220) :int(end*fs-330) ] = 1
        word =np.append(word,np.zeros((l-n)))
        word_f=110
        for i in range(len(audio)//word_f):
            data = word[int(i*word_f):int((i+1)*word_f)]
            if sum(data==0)!=0:
                frame_word.append(0)
                frame_time.append((i*word_f)/fs)
            else:
                frame_word.append(1)
                frame_time.append((i*word_f)/fs)

        # adding -1 to word break-------------------------------------------
        for i in range(len(frame_word)-1):
          if frame_word[i+1]!=frame_word[i]:
                frame_word[i]=-1
        frame_word = np.array(frame_word)
        add_1_to_word = np.append([False],np.array(frame_word==-1)[:-1])
        frame_word[add_1_to_word] = -1
        # adding -1 to word break-----------------------------------------------
        amp=audio
        window = 0.005 # in ms
        S = librosa.feature.melspectrogram(y=audio.astype('float'), sr=fs, n_mels=2*128,fmax=8000,hop_length=int(fs*window),n_fft=1024)
        X = np.flipud(librosa.power_to_db(S)) 
        frame= 100 # 200 mean 1 sec
        s_frame = 100 # 200 mean 1 sec
        for i in range((l//(fs//2))):
            train_x.append(X[:,int(s_frame*i):int(s_frame*(i+1))])
            train_y.append(frame_word[int(frame*i):int(frame*(i+1))])
    except Exception as e:
      print('pass', e)
      break
     
np.save(datadir +'X',train_x)
np.save(datadir +'y',train_y)

In [0]:
train_x = np.load(datadir+'X.npy',allow_pickle = True)
train_y = np.load(datadir+'y.npy',allow_pickle = True)

In [33]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.keras.layers import Input,Dense,Flatten,Dropout,Conv2D,BatchNormalization,Reshape,LSTM,MaxPooling2D,LeakyReLU
from tensorflow.keras.models import Model,Sequential,load_model
from tensorflow.keras.callbacks import ModelCheckpoint

TensorFlow 2.x selected.


In [0]:
model = Sequential()

model.add(Conv2D(64,(3,3),padding='same',input_shape=(256,100,1)))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(Dropout(0.25))


model.add(Conv2D(32,(3,3),padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(Dropout(0.25))


model.add(Conv2D(16,(3,3),padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(Dropout(0.25))


model.add(Conv2D(8,(3,3),padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(Dropout(0.25))

model.add(Conv2D(4,(3,3),padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(512,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(256,activation='relu'))
model.add(Dense(100,activation='sigmoid'))



In [44]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 256, 100, 64)      640       
_________________________________________________________________
batch_normalization_9 (Batch (None, 256, 100, 64)      256       
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 256, 100, 64)      0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 256, 100, 64)      0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 256, 100, 32)      18464     
_________________________________________________________________
batch_normalization_10 (Batc (None, 256, 100, 32)      128       
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 256, 100, 32)     

In [0]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [0]:
import sys
X=[]
y=[]
training_data = list(zip(train_x,train_y))
np.random.shuffle(training_data)
for i in training_data:
    r = len(i[1])
    if r==100:
            X.append(i[0])
            y.append(i[1])
X = np.array(X).reshape(-1,256,100,1)


X = (X-np.mean(X))/np.std(X) # normalize the data
y = np.array(y).reshape(-1,100)

In [0]:
model_name = datadir+'word_boundary-ver-1.0.model' 
callback = ModelCheckpoint(model_name,monitor='val_loss', save_best_only=True ,mode='min')

In [0]:
model.fit(X,y,batch_size=1,epochs=10,validation_split=0.3,callbacks=[callback])

In [0]:
def pre_prosessing(audio,fs=22050):
    slot=[]
    n = len(audio)
    l = (((n//(fs//2))+1)*(fs//2))
    audio =np.append(audio,np.zeros((l-n)))
    time  = np.arange(0,len(audio)/fs,1/fs)
    m = min(len(time),len(audio))
    window = 0.005
    S = librosa.feature.melspectrogram(y=audio.astype('float'), sr=fs, n_mels=2*128,fmax=8000,hop_length=int(fs*window),n_fft=1024)
    X = np.flipud(librosa.power_to_db(S))
    s_frame = 100 
    join_predictions=[]
    for i in range((l//(fs//2))):
          slot.append(X[:,int(s_frame*i):int(s_frame*(i+1))])
    slot = np.array(slot).reshape(-1,256,100,1)
    slot = (slot-np.mean(slot))/np.std(slot)
    for i in range(slot.shape[0]):
        join_predictions.append(model.predict(slot[i:i+1])[0])
    join_predictions = np.array(join_predictions).reshape(1,-1)
    return join_predictions[0]


TEST THE MODEL

In [0]:
#test
file_name ='s10_f1_a1' #add the file name
fs,audio = read(datadir_waves + file_name +'.wav') 
df = pd.read_csv(datadir_cutout+file_name+'.csv')
time  = np.arange(0,len(audio)/fs,1/fs)
m = min(len(time),len(audio))


result = pre_prosessing(audio)
pred_time = np.linspace(0,((len(result)//100)*(fs/2))/fs,(len(result)//100)*100)
  
 
fig = plt.figure(figsize=(17,7))
ax = fig.add_subplot(111)
ax.plot(time[:m],(audio/np.max(audio))[:m])          
ax.plot(pred_time,np.array(result),label = 'predicted')

ax.legend()


  

